In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing Libraries


In [ ]:
import numpy as np 
import pandas as pd 

import os

import matplotlib.pylab as plt
import seaborn as sns

import librosa
import librosa.display
import librosa.effects as le
import IPython.display as ipd

Load the Dataset


In [ ]:
Tess = "/kaggle/input/speech-emotion-recognition-en/Tess"
tess_directory_list = os.listdir(Tess)


labels = []
paths = []

for dir in tess_directory_list:
    directories = os.listdir(Tess +'/'+ dir)
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[-1]
        if(part=="ps"):
           labels.append("surprise")
        else:
           labels.append(part)
        paths.append(Tess +'/'+ dir + '/' + file)
       
# dataframe for emotion of files
emotion_df = pd.DataFrame(labels, columns=['Emotions'])
path_df = pd.DataFrame(paths, columns=['Path'])
Tess_df = pd.concat([emotion_df, path_df], axis=1)
Tess_df.head()

In [ ]:
paths[0]

In [ ]:
labels[:5]

In [ ]:
df = pd.DataFrame();
df['speech'] = paths
df['label'] = labels
df.head()

In [ ]:
# df['label'].value_counts()
df['speech']

Counting the labels

In [ ]:
sns.countplot(data=df, x='label')
plt.show()

**plotting waveform**

In [ ]:
def waveplot(data,sr,emotion):
    plt.figure(figsize=(10,4))
    plt.title(emotion,size=20)
    librosa.display.waveshow(data,sr=sr)
    plt.show()

In [ ]:
emotion = 'fear'
path = np.array(df['speech'][df['label']==emotion])[0]
data,sr = librosa.load(path)
waveplot(data,sr,emotion)

plotting spectogram

In [ ]:
def spectogram(data,sr,emotion):
   x = librosa.stft(data)
   xdb = librosa.amplitude_to_db(abs(x))
   plt.figure(figsize=(10,4))
   plt.title(emotion,size=20)
   librosa.display.specshow(xdb,sr=sr,x_axis='time',y_axis='hz')
   plt.colorbar()
emotion = 'surprise'
path = np.array(df['speech'][df['label']==emotion])[0]
data,sr = librosa.load(path)
spectogram(data,sr,emotion)

In [ ]:
def spectogram(data,sr,emotion):
   x = librosa.stft(data)
   xdb = librosa.amplitude_to_db(abs(x))
   plt.figure(figsize=(10,4))
   plt.title(emotion,size=20)
   librosa.display.specshow(xdb,sr=sr,x_axis='time',y_axis='hz')
   plt.colorbar()
emotion = 'fear'
path = np.array(df['speech'][df['label']==emotion])[0]
data,sr = librosa.load(path)
spectogram(data,sr,emotion)

In [ ]:
def spectogram(data,sr,emotion):
   x = librosa.stft(data)
   xdb = librosa.amplitude_to_db(abs(x))
   plt.figure(figsize=(10,4))
   plt.title(emotion,size=20)
   librosa.display.specshow(xdb,sr=sr,x_axis='time',y_axis='hz')
   plt.colorbar()
emotion = 'angry'
path = np.array(df['speech'][df['label']==emotion])[0]
data,sr = librosa.load(path)
spectogram(data,sr,emotion)

In [ ]:
def spectogram(data,sr,emotion):
   x = librosa.stft(data)
   xdb = librosa.amplitude_to_db(abs(x))
   plt.figure(figsize=(10,4))
   plt.title(emotion,size=20)
   librosa.display.specshow(xdb,sr=sr,x_axis='time',y_axis='hz')
   plt.colorbar()
emotion = 'happy'
path = np.array(df['speech'][df['label']==emotion])[0]
data,sr = librosa.load(path)
spectogram(data,sr,emotion)

In [ ]:
def spectogram(data,sr,emotion):
   x = librosa.stft(data)
   xdb = librosa.amplitude_to_db(abs(x))
   plt.figure(figsize=(10,4))
   plt.title(emotion,size=20)
   librosa.display.specshow(xdb,sr=sr,x_axis='time',y_axis='hz')
   plt.colorbar()
emotion = 'surprise'
path = np.array(df['speech'][df['label']==emotion])[0]
data,sr = librosa.load(path)
spectogram(data,sr,emotion)

Feature Extraction

In [ ]:
def extract_mfcc(fn):
    y,sr = librosa.load(fn,duration=3,offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)
    return mfcc

In [ ]:
extract_mfcc(paths[0])

In [ ]:
x_mfcc = df['speech'].apply(lambda x: extract_mfcc(x
                                                  ))

In [ ]:
x_mfcc

In [ ]:
X = [x for x in x_mfcc]
X = np.array(X)
X.shape

In [ ]:
# X = np.expand_dims(X,-1)
# X.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']])

In [ ]:
y = y.toarray()

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential([
    LSTM(123, return_sequences=False, input_shape=(40, 1)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential([
    LSTM(123, return_sequences=False, input_shape=(40, 1)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model_cnn = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(40, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])

model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.summary()


In [ ]:
history = model_cnn.fit(X,y,validation_split=0.2,epochs=100,batch_size=512,shuffle=True)

In [ ]:
epochs = list(range(100))
loss = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs,loss,label = 'accuracy')
plt.plot(epochs,val_acc,label = 'val_accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# Generate a synthetic dataset for demonstration
# X, y = make_classification(n_samples=1000, n_features=40, n_informative=20, n_classes=7, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=100)

# Standardize features (important for KNN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a KNN classifier with, for example, k=3
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Train the classifier
knn_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# Load the Iris dataset
# iris = load_iris()
# X, y = iris.data, iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (important for KNN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Test different values of k (number of neighbors)
for k in range(1, 11):
    # Create a KNN classifier with the current value of k
    knn_classifier = KNeighborsClassifier(n_neighbors=k)

    # Train the classifier
    knn_classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = knn_classifier.predict(X_test)

    # Evaluate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for k={k}: {accuracy:.2f}")


In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

# Generate a synthetic dataset for demonstration
# In a real-world scenario, you would replace this with your own dataset
# X, y = make_classification(n_samples=1000, n_features=40, n_informative=20, n_classes=7, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (important for KNN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Train the classifier
knn_classifier.fit(X_train, y_train)

# Generate random MFCCs for a single audio sample
random_mfcc = np.random.rand(40).reshape(1, -1)

# Standardize the random MFCC using the same scaler
random_mfcc_standardized = scaler.transform(random_mfcc)

# Predict the emotion label for the random MFCC
predicted_emotion = knn_classifier.predict(random_mfcc_standardized)

print(f"Predicted Emotion Label: {predicted_emotion[0]}")


In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

# Generate a synthetic dataset for demonstration
# In a real-world scenario, you would replace this with your own dataset
# X, y = make_classification(n_samples=1000, n_features=40, n_informative=20, n_classes=7, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (important for KNN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Train the classifier
knn_classifier.fit(X_train, y_train)

# Generate random MFCCs for a single audio sample
random_mfcc = np.random.rand(40).reshape(1, -1)

# Standardize the random MFCC using the same scaler
random_mfcc_standardized = scaler.transform(random_mfcc)

# Predict the emotion label for the random MFCC
predicted_emotion = knn_classifier.predict(random_mfcc_standardized)

# Generate a random actual emotion label for demonstration
actual_emotion = np.random.randint(7)  # Assuming 7 different emotions

print(f"Actual Emotion Label: {actual_emotion}")
print(f"Predicted Emotion Label: {predicted_emotion[0]}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

# Generate a synthetic dataset for demonstration
# In a real-world scenario, you would replace this with your own dataset
# X, y = make_classification(n_samples=1000, n_features=40, n_informative=20, n_classes=7, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (important for KNN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Train the classifier
knn_classifier.fit(X_train, y_train)

# Generate 50 random MFCCs and predict their emotions
num_samples = 50
random_mfccs = np.random.rand(num_samples, 40)
random_mfccs_standardized = scaler.transform(random_mfccs)
predicted_emotions = knn_classifier.predict(random_mfccs_standardized)

# Generate random actual emotion labels for demonstration
actual_emotions = np.random.randint(7, size=num_samples)  # Assuming 7 different emotions

# Calculate accuracy percentage
accuracy_percentage = accuracy_score(actual_emotions, predicted_emotions) * 100

# Print actual and predicted emotions
print("Actual Emotion Labels:", actual_emotions)
print("Predicted Emotion Labels:", predicted_emotions)

# Print accuracy percentage
print(f"Accuracy Percentage: {accuracy_percentage:.2f}%")

# Plot a bar graph of actual vs predicted emotions
fig, ax = plt.subplots()
bar_width = 0.35
index = np.arange(num_samples)
bar1 = ax.bar(index, actual_emotions, bar_width, label='Actual Emotions')
bar2 = ax.bar(index + bar_width, predicted_emotions, bar_width, label='Predicted Emotions')

ax.set_xlabel('Sample Index')
ax.set_ylabel('Emotion Labels')
ax.set_title('Actual vs Predicted Emotions')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(index)
ax.legend()

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

# Generate a synthetic dataset for demonstration
# In a real-world scenario, you would replace this with your own dataset
# X, y = make_classification(n_samples=1000, n_features=40, n_informative=20, n_classes=7, random_state=42)

# Standardize features (important for KNN)
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

# Create a KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Train the classifier
knn_classifier.fit(X_standardized, y)

# Generate 50 random MFCCs and predict their emotions
num_samples = 20
random_mfccs = np.random.rand(num_samples, 40)
random_mfccs_standardized = scaler.transform(random_mfccs)
predicted_emotions = knn_classifier.predict(random_mfccs_standardized)

# Generate random actual emotion labels for demonstration
actual_emotions = np.random.randint(7, size=num_samples)  # Assuming 7 different emotions

# Calculate accuracy for each sample
accuracy_per_sample = [accuracy_score([actual], [predicted]) for actual, predicted in zip(actual_emotions, predicted_emotions)]

# Plot a bar graph of accuracy for each sample
fig, ax = plt.subplots()
index = np.arange(num_samples)
bar_width = 0.35

bars = ax.bar(index, accuracy_per_sample, bar_width, label='Accuracy per Sample')

ax.set_xlabel('Sample Index')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy for Each Sample')
ax.set_xticks(index)
ax.legend()

plt.show()


In [ ]:
history = knn_classifier.fit(X,y,validation_split=0.2,epochs=0,batch_size=512,shuffle=True)